In [1]:
import os
import random
from moviepy.editor import *
import moviepy.audio.fx.all as afx
from moviepy.video import fx
from moviepy.config import change_settings
change_settings({"FFMPEG_BINARY":"ffmpeg"})
# from PIL import Image

In [2]:
select_number_of_news = int(input('Select news: '))

In [3]:
select_lang = int(input('Select language: '))

if select_lang == 1:
    folder_lang = 'english'
    lang = 'en'
elif select_lang == 2:
    folder_lang = 'spanish'
    lang = 'es'
else:
    pass

In [4]:
# Configuración
carpeta1 = f'/mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/3_extract_img_videos/img_by_number/{select_number_of_news}'
carpeta2 = f'/mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/3_extract_img_videos/video_by_number/{select_number_of_news}'
audio_musica = f'/mnt/D8E84E4DE84E2A58/backUp_Img_Vid/Intro/History/New_metal.mp3'   # Ruta específica del audio de música
audio_voz = f'/mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/1_create_first_audio/output_{lang}/{select_number_of_news}/audio_final_{select_number_of_news}.wav' 

In [5]:
# Función para seleccionar archivos aleatorios de cada carpeta
def selecciona_archivos(carpeta, num_archivos=25):
    archivos = [f for f in os.listdir(carpeta) if f.endswith(('.jpg','.png', '.gif', '.mp4', '.avi'))]
    if len(archivos) == 0:
        raise ValueError(f'La carpeta {carpeta} está vacía o no contiene archivos válidos.')
    return random.sample(archivos, min(num_archivos, len(archivos)))

In [6]:
# Selección de archivos aleatorios para cada carpeta
carpeta1_archivos = selecciona_archivos(carpeta1)
carpeta2_archivos = selecciona_archivos(carpeta2)

In [7]:
# Obtener duración del audio principal
audio_principal_voz = AudioFileClip(audio_voz)
duracion_audio_principal = audio_principal_voz.duration

In [8]:
# Crear una lista para almacenar los clips seleccionados
clips_seleccionados = []

# Obtener todos los archivos de las carpetas
archivos = carpeta1_archivos + carpeta2_archivos
random.shuffle(archivos)  # Mezclar los archivos para alternar entre fotos y videos

# Procesar los archivos seleccionados y añadirlos como clips
for archivo in archivos:
    archivo_path = os.path.join(carpeta1 if archivo in carpeta1_archivos else carpeta2, archivo)
    
    if archivo.endswith(('.jpg', '.png')):
        # Agregar foto al video con duración aleatoria
        try:
            # Redimensionar imagen a 1920x1080 usando MoviePy
            foto = ImageClip(archivo_path).set_duration(random.uniform(2, 5)).resize((1920, 1080))
            if not hasattr(foto, 'duration') or foto.duration is None:
                foto = foto.set_duration(10)  # Asignar duración predeterminada si es necesario
            clips_seleccionados.append(foto)
        except Exception as e:
            print(f"Error al procesar la imagen {archivo}: {e}")
    
    elif archivo.endswith(('.mp4', '.avi')):
        # Agregar video al video con duración ajustada
        try:
            # Redimensionar video a 1920x1080 usando MoviePy
            clip = VideoFileClip(archivo_path).resize((1920, 1080))
            clip_duracion = min(random.uniform(3, 7), clip.duration)
            if not hasattr(clip, 'duration') or clip.duration is None:
                clip = clip.set_duration(clip_duracion)
            clips_seleccionados.append(clip)
        except Exception as e:
            print(f"Error al procesar el video {archivo}: {e}")

In [9]:
# Cargar los clips de audio
try:
    # Cargar los clips de audio
    audio_principal_voz = AudioFileClip(audio_voz)
    audio_fondo = AudioFileClip(audio_musica)

    # Repetir la música de fondo para que tenga la misma duración que el audio de voz
    if audio_fondo.duration < audio_principal_voz.duration:
        # Calcular cuántas veces repetir la música de fondo
        n_repeats = int(audio_principal_voz.duration // audio_fondo.duration) + 1
        audio_fondo_repetida = concatenate_audioclips([audio_fondo] * n_repeats)
        # Ajustar la duración
        audio_fondo_repetida = audio_fondo_repetida.subclip(0, audio_principal_voz.duration)
    else:
        # Ajustar la duración si es más larga que el audio de voz
        audio_fondo_repetida = audio_fondo.subclip(0, audio_principal_voz.duration)

    # Ajustar volúmenes
    audio_fondo_repetida = audio_fondo_repetida.volumex(0.1)  # Bajar volumen de fondo
    audio_principal_voz = audio_principal_voz.volumex(1.8)    # Subir volumen de voz

    # Combina los audios de voz y música (ajusta si es necesario)
    final_audio = CompositeAudioClip([audio_fondo_repetida, audio_principal_voz])
    
except Exception as e:
    print(f"Error al combinar los audios: {e}")
    final_audio = audio_principal_voz

In [10]:
# Verificar si la suma de las duraciones de los clips es menor a la duración del audio principal
suma_duracion_clips = sum([clip.duration for clip in clips_seleccionados])

# Función para seleccionar y procesar clips, priorizando mp4
def seleccionar_y_procesar_clips(carpeta1_archivos, carpeta2_archivos, carpeta1, carpeta2, duracion_restante):
    nuevos_clips = []
    
    # Priorizar archivos .mp4
    archivos_prioritarios = [archivo for archivo in carpeta1_archivos + carpeta2_archivos 
                             if archivo.endswith('.mp4')]
    
    # Luego, considerar los demás archivos
    archivos_restantes = [archivo for archivo in carpeta1_archivos + carpeta2_archivos
                          if not archivo.endswith('.mp4')]
    
    # Mezclar ambos conjuntos de archivos
    random.shuffle(archivos_prioritarios)
    random.shuffle(archivos_restantes)
    
    # Procesar primero los archivos prioritarios
    for archivo in archivos_prioritarios:
        archivo_path = os.path.join(carpeta1 if archivo in carpeta1_archivos else carpeta2, archivo)

        try:
            # Procesar archivos .mp4 (videos)
            clip = VideoFileClip(archivo_path).resize((1920, 1080))
            clip_duracion = min(random.uniform(3, 7), clip.duration)
            if not hasattr(clip, 'duration') or clip.duration is None:
                clip = clip.set_duration(clip_duracion)
            nuevos_clips.append(clip)
        except Exception as e:
            print(f"Error al procesar el video {archivo}: {e}")

        # Si la duración total alcanza o supera la duración restante, detener la selección
        if sum([clip.duration for clip in nuevos_clips]) >= duracion_restante:
            return nuevos_clips
    
    # Procesar archivos restantes si aún hay espacio para más clips
    for archivo in archivos_restantes:
        archivo_path = os.path.join(carpeta1 if archivo in carpeta1_archivos else carpeta2, archivo)

        if archivo.endswith(('.jpg', '.png')):
            # Agregar foto al video con duración aleatoria
            try:
                foto = ImageClip(archivo_path).set_duration(random.uniform(2, 5)).resize((1920, 1080))
                if not hasattr(foto, 'duration') or foto.duration is None:
                    foto = foto.set_duration(3)
                nuevos_clips.append(foto)
            except Exception as e:
                print(f"Error al procesar la imagen {archivo}: {e}")

        elif archivo.endswith(('.avi', '.gif')):
            # Agregar video al video con duración ajustada
            try:
                clip = VideoFileClip(archivo_path).resize((1920, 1080))
                clip_duracion = min(random.uniform(3, 7), clip.duration)
                if not hasattr(clip, 'duration') or clip.duration is None:
                    clip = clip.set_duration(clip_duracion)
                nuevos_clips.append(clip)
            except Exception as e:
                print(f"Error al procesar el video {archivo}: {e}")
        
        if sum([clip.duration for clip in nuevos_clips]) >= duracion_restante:
            break

    return nuevos_clips

# Mientras la suma de las duraciones de los clips sea menor que la duración del audio principal
while suma_duracion_clips < duracion_audio_principal:
    # Calcular la duración restante
    duracion_restante = duracion_audio_principal - suma_duracion_clips
    
    # Seleccionar nuevas imágenes y videos aleatoriamente
    nuevos_clips = seleccionar_y_procesar_clips(carpeta1_archivos, carpeta2_archivos, carpeta1, carpeta2, duracion_restante)
    
    # Añadir los nuevos clips a la lista de clips seleccionados
    clips_seleccionados.extend(nuevos_clips)
    
    # Actualizar la suma de las duraciones de los clips
    suma_duracion_clips = sum([clip.duration for clip in clips_seleccionados])

# Ajustar la duración total para que coincida exactamente con la duración del audio principal
factor_ajuste = duracion_audio_principal / suma_duracion_clips
clips_seleccionados = [clip.set_duration(clip.duration * factor_ajuste) for clip in clips_seleccionados]


In [11]:
# Concatenar los clips y crear el video final
try:
    final_video = concatenate_videoclips(clips_seleccionados, method="compose").set_audio(final_audio)
except Exception as e:
    print(f"Error al concatenar los clips: {e}")
    raise

In [12]:
# Verificar y asignar FPS si no está definido
if not hasattr(final_video, 'fps') or final_video.fps is None:
    final_video = final_video.set_fps(24)  # Asignar un valor predeterminado de FPS

In [13]:
# Guardar el video automático
try:
    final_video.write_videofile(f'videos_by_news/{select_number_of_news}.mp4', codec="h264_nvenc", audio_codec="aac", fps=24)
    # final_video.write_videofile(f'videos_by_news/{select_number_of_news}.mp4', codec="libx264", audio_codec="aac", fps=24)
    print("¡Listo! Video automático creado con éxito.")
except Exception as e:
    print(f"Error al guardar el video: {e}")

Moviepy - Building video videos_by_news/7.mp4.
MoviePy - Writing audio in 7TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video videos_by_news/7.mp4



Moviepy - Done !
Moviepy - video ready videos_by_news/7.mp4
¡Listo! Video automático creado con éxito.
